In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.01
seed = 0
alpha = 1e-2
lamda = 1e-3
eta = 1e-2
d0 = 2
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.2        0.         0.2        0.         0.
  0.25       0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.2        0.         0.         0.         0.125      0.
  0.         0.         0.2        0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.14285714]
 [0.         0.         0.         0.         0.125      0.
  0.2        0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.14285714]
 [0.2        0.         0.         0.         0.125      0.2
  0.         0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.125      0.125      0.125      0.         0.125
  0.         0.125      0.125      0.         0.         0.
  0.         0.         0.         0.         0.        

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0])
W2 = np.array([-2.0, 2.0])
W = [W1, W2]
m = 200
n = 2
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(8.8291, grad_fn=<MseLossBackward0>) tensor([ 4.9329, -3.2493]) tensor([0.4254, 0.2476], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(7.2305, grad_fn=<MseLossBackward0>) tensor([ 3.1584, -4.0442]) tensor([0.3761, 0.2801], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(6.0486, grad_fn=<MseLossBackward0>) tensor([ 3.2629, -2.6477]) tensor([0.3445, 0.3205], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(6.4321, grad_fn=<MseLossBackward0>) tensor([ 4.1023, -2.0447]) tensor([0.3119, 0.3470], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(7.7148, grad_fn=<MseLossBackward0>) tensor([ 4.6188, -3.0266]) tensor([0.2709, 0.3674], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(6.3080, grad_fn=<MseLossBackward0>) tensor([ 2.9581, -3.7663]) tensor([0.2247, 0.3977], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50

16 tensor(0.7507, grad_fn=<MseLossBackward0>) tensor([ 1.4422, -0.6395]) tensor([-1.1966,  1.4642], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(0.8927, grad_fn=<MseLossBackward0>) tensor([ 1.6060, -0.9795]) tensor([-1.2111,  1.4706], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(0.7147, grad_fn=<MseLossBackward0>) tensor([ 1.0324, -1.2150]) tensor([-1.2271,  1.4804], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(0.6080, grad_fn=<MseLossBackward0>) tensor([ 1.0706, -0.7872]) tensor([-1.2374,  1.4925], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
17 tensor(0.6564, grad_fn=<MseLossBackward0>) tensor([ 1.3505, -0.5951]) tensor([-1.2482,  1.5004], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
18 tensor(0.7802, grad_fn=<MseLossBackward0>) tensor([ 1.5031, -0.9132]) tensor([-1.2617,  1.5063], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
18 tensor(0.6240

33 tensor(0.0767, grad_fn=<MseLossBackward0>) tensor([ 0.4703, -0.1898]) tensor([-1.7403,  1.8359], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0905, grad_fn=<MseLossBackward0>) tensor([ 0.5196, -0.2991]) tensor([-1.7450,  1.8378], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0714, grad_fn=<MseLossBackward0>) tensor([ 0.3350, -0.3705]) tensor([-1.7502,  1.8408], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0614, grad_fn=<MseLossBackward0>) tensor([ 0.3481, -0.2382]) tensor([-1.7535,  1.8445], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
34 tensor(0.0670, grad_fn=<MseLossBackward0>) tensor([ 0.4402, -0.1768]) tensor([-1.7570,  1.8469], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
35 tensor(0.0791, grad_fn=<MseLossBackward0>) tensor([ 0.4862, -0.2791]) tensor([-1.7614,  1.8487], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
35 tensor(0.0623

51 tensor(0.0062, grad_fn=<MseLossBackward0>) tensor([ 0.1124, -0.0732]) tensor([-1.9207,  1.9518], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
51 tensor(0.0068, grad_fn=<MseLossBackward0>) tensor([ 0.1425, -0.0535]) tensor([-1.9219,  1.9526], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
52 tensor(0.0080, grad_fn=<MseLossBackward0>) tensor([ 0.1566, -0.0859]) tensor([-1.9233,  1.9531], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
52 tensor(0.0063, grad_fn=<MseLossBackward0>) tensor([ 0.1012, -0.1067]) tensor([-1.9248,  1.9540], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
52 tensor(0.0054, grad_fn=<MseLossBackward0>) tensor([ 0.1052, -0.0684]) tensor([-1.9259,  1.9550], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
52 tensor(0.0060, grad_fn=<MseLossBackward0>) tensor([ 0.1333, -0.0499]) tensor([-1.9269,  1.9557], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
53 tensor(0.0070

torch.Size([50, 1])
torch.Size([50, 1])
70 tensor(0.0007, grad_fn=<MseLossBackward0>) tensor([ 0.0471, -0.0246]) tensor([-1.9771,  1.9863], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
70 tensor(0.0006, grad_fn=<MseLossBackward0>) tensor([ 0.0305, -0.0311]) tensor([-1.9775,  1.9865], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
70 tensor(0.0005, grad_fn=<MseLossBackward0>) tensor([ 0.0316, -0.0200]) tensor([-1.9778,  1.9868], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
70 tensor(0.0005, grad_fn=<MseLossBackward0>) tensor([ 0.0401, -0.0143]) tensor([-1.9781,  1.9870], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
71 tensor(0.0006, grad_fn=<MseLossBackward0>) tensor([ 0.0441, -0.0230]) tensor([-1.9785,  1.9872], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
71 tensor(0.0005, grad_fn=<MseLossBackward0>) tensor([ 0.0286, -0.0291]) tensor([-1.9790,  1.9874], grad_fn=<CatBackward0>)
torch.Size([50, 

89 tensor(4.6286e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0086, -0.0086]) tensor([-1.9938,  1.9963], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
89 tensor(4.0388e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0088, -0.0057]) tensor([-1.9939,  1.9963], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
89 tensor(4.3930e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0112, -0.0039]) tensor([-1.9940,  1.9964], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(4.9590e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0124, -0.0059]) tensor([-1.9941,  1.9964], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(4.0846e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0081, -0.0080]) tensor([-1.9942,  1.9965], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
90 tensor(3.5654e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0082, -0.0054]) tensor([-1.9943,  1.9966], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([

In [11]:
parameters_to_vector(model.parameters())

tensor([-1.9970,  1.9982], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, 0, tensor([[0.4676, 0.0000],
        [0.0000, 0.4676]]), tensor([[0.0884, 0.0000],
        [0.0000, 0.0884]]), 0, tensor([[2.1782, 0.0000],
        [0.0000, 2.1782]]), tensor([[0.1747, 0.0000],
        [0.0000, 0.1747]]), 0, tensor([[1.2504, 0.0000],
        [0.0000, 1.2504]]), tensor([[-0.2033,  0.0000],
        [ 0.0000, -0.2033]]), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(2.8284, dtype=torch.float64)

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([0.5557, 0.1696])


tensor([1.1113, 0.3392], grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros(d0)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(projection_list[i][k], parameters_to_vector(models[i].parameters())) - torch.matmul(projection_list[k][i], parameters_to_vector(models[k].parameters())))
            temp_mat = torch.outer(mat_vec_sum, weights).clone()


            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 1/2000 [00:00<04:22,  7.61it/s]

Training_loss 9.17849,   Relative Error 1.14337


  0%|          | 2/2000 [00:00<03:53,  8.54it/s]

Training_loss 8.85918,   Relative Error 1.12412


  0%|          | 3/2000 [00:00<03:49,  8.69it/s]

...

100%|█████████▉| 1997/2000 [04:09<00:00, 10.16it/s]

Training_loss 0.00004,   Relative Error 0.00231
Training_loss 0.00004,   Relative Error 0.00229
Training_loss 0.00004,   Relative Error 0.00230


100%|██████████| 2000/2000 [04:09<00:00, 10.10it/s]

Training_loss 0.00004,   Relative Error 0.00232
Training_loss 0.00004,   Relative Error 0.00231
Training_loss 0.00004,   Relative Error 0.00230


100%|██████████| 2000/2000 [04:09<00:00,  8.01it/s]


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-2.0049,  2.0042], grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

2
3
5
6
8
9


In [23]:
parameters_to_vector(models[0].parameters())

tensor([2.0010, 2.0011], grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 0,
 tensor([[0.5709, 0.1028],
         [0.0927, 0.5597]], grad_fn=<AddBackward0>),
 tensor([[0.1916, 0.1028],
         [0.0927, 0.1805]], grad_fn=<AddBackward0>),
 0,
 tensor([[2.2814, 0.1028],
         [0.0927, 2.2703]], grad_fn=<AddBackward0>),
 tensor([[0.2779, 0.1027],
         [0.0927, 0.2668]], grad_fn=<AddBackward0>),
 0,
 tensor([[1.3536, 0.1027],
         [0.0927, 1.3425]], grad_fn=<AddBackward0>),
 tensor([[-0.1000,  0.1027],
         [ 0.0927, -0.1111]], grad_fn=<AddBackward0>),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [25]:
projected_weights[0]

[0,
 0,
 tensor([1.3479, 1.3056]),
 tensor([0.5891, 0.5467]),
 0,
 tensor([4.7708, 4.7287]),
 tensor([0.7618, 0.7194]),
 0,
 tensor([2.9142, 2.8720]),
 tensor([ 0.0054, -0.0370]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
print(test_loss)

[9.17848790e+00 8.85918307e+00 8.47640347e+00 ... 3.83748611e-05
 3.82949083e-05 3.79398261e-05]


In [28]:
np.save( 'training_loss_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)

In [31]:
'training_loss_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'training_loss_sheave_fml0_01_pout0_01'

In [32]:
'relative_error_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'relative_error_sheave_fml0_01_pout0_01'